# Gods, demons, and mortals
Inspired by Raymond Smullyan's "Gods, fter deamons, and mortals" puzzles from *To Mock a Mockingbird, and other logic puzzles*. These puzzles are similar to the familiar "knights and knave" puzzles.
> Shortly after Inspector Craig returned to London from his strange experience in Subterranea, he had a curious dream... Craig dreamed he spent nine days in a region in which dwelled gods, deamons and mortals. The gods of course, always told the truth, and the demons always lied. As to the mortals half were knights and half were knaves. As usual, the knights told the truth and the knaves lied.

In [1]:
#there are 4 kinds of inhabitants: gods, knights, knaves and demons
options = ['god', 'knight', 'knave', 'demon']


In [2]:
mortals = ['knight','knave']
truthfuls =['god','knight']

In [3]:
#let's have puzzles involving pairs of inhabitants - there are 16 possible pairs
pairs = []
for i in options:
    for j in options:
        pairs.append([i,j])
print(pairs)

[['god', 'god'], ['god', 'knight'], ['god', 'knave'], ['god', 'demon'], ['knight', 'god'], ['knight', 'knight'], ['knight', 'knave'], ['knight', 'demon'], ['knave', 'god'], ['knave', 'knight'], ['knave', 'knave'], ['knave', 'demon'], ['demon', 'god'], ['demon', 'knight'], ['demon', 'knave'], ['demon', 'demon']]


In [4]:
#lets figure out all the way 'A' could be lying, and all the ways 'B' could be lying
aLying = []
bLying = []
for i in options:
    aLying.append(['demon',i])
    aLying.append(['knave',i])
    bLying.append([i,'demon'])
    bLying.append([i,'knave'])

In [5]:
#let's have a function that creates the sets of pairs based on two lists
def fromLists(a,b):
    res = []
    for i in a:
        for j in b:
            res.append([i,j])
    return res

In [6]:
# return the list that is the intersection of two lists
def intersect(a, b):
    return [item for item in a if item in b]
    
# return the list that is the union of two lists
def union(a, b):
    return list(a) + [item for item in b if item not in a]

# return the list that is the complement of two lists (based on the big list of pairs)
def complement(a):
    return [item for item in pairs if item not in a]

# return list a - list b
def difference(a,b):
    return [item for item in a if item not in b]

In [7]:
# now from mortals, we can define the immortals
immortals = difference(options,mortals)
print(immortals)

['god', 'demon']


In [8]:
# and from the truth tellers, we can find the liars
liars = difference(options, truthfuls)
print(liars)

['knave', 'demon']


In [9]:
# we introduce the idea of 'sides' and of 'dimensions'
def sameSide(x):
    if (x == 'god' or x=='knight'):
        return truthfuls
    return liars
print(sameSide('god'))

def sameDimension(x):
    if (x=='god' or x== 'demon'):
        return immortals
    return mortals
print(sameDimension('knight'))

['god', 'knight']
['knight', 'knave']


In [10]:
aTruthing = complement(aLying)
bTruthing = complement(bLying)

In [11]:
# aX(value) returns all pairs where 'A' is value
def aX(value):
    res = []
    for i in options:
        res.append([value, i])
    return res

#bX(value) returns all pairs where 'B' is value
def bX(value):
    res=[]
    for i in options:
        res.append([i,value])
    return res

The code below constructs the situation where A says 'I am a knave'.
1. A could be telling the truth: aTruths is the intersection of all cases of A being a knave with A telling the truth. (should be empty)
2. A could be lying. aLies is the intersection of all cases of A not being a knave and A lying.

Turns out, A must be a demon.

In [12]:
aTruths = intersect(aX('knave'),aTruthing)
aLies = intersect(complement(aX('knave')),aLying)
result1 = union(aLies, aTruths)
print(aTruths)
print(result1)


[]
[['demon', 'god'], ['demon', 'knight'], ['demon', 'knave'], ['demon', 'demon']]


The code below constructs the situation where B says 'we are both demons'

In [13]:
bothDemons =[['demon','demon']]
bTruths = intersect(bothDemons, bTruthing)
bLies = intersect(complement(bothDemons), bLying)
result2 = union(bTruths,bLies)
print(bTruths)
print(result2)                               

[]
[['god', 'knave'], ['god', 'demon'], ['knight', 'knave'], ['knight', 'demon'], ['knave', 'knave'], ['knave', 'demon'], ['demon', 'knave']]


In [14]:
print(intersect(result1,result2))

[['demon', 'knave']]


In [15]:
# Let's have functions that create the states of affairs arising from A's statements, B's statements, 
# and then the solution must be in the intersection of the two.

def aPossible(aStatement):
    aTruths = intersect(aStatement, aTruthing)
    aLies = intersect(complement(aStatement),aLying)
    return union(aTruths, aLies)

def bPossible(bStatement): 
    bTruths = intersect(bStatement, bTruthing)
    bLies = intersect(complement(bStatement),bLying)
    return union(bTruths, bLies)

def solutions(aStatement, bStatement):
    return intersect(aPossible(aStatement),bPossible(bStatement))


## Some possible puzzle scenarios

In [16]:
# A says they are a knave, and B says they are both demons
solutions(aX('knave'),[['demon','demon']])

[['demon', 'knave']]

In [17]:
# A says they are both demons, and B says they are both demons
solutions(complement(['demon','demon']),[['demon','demon']])

[['god', 'knave'], ['god', 'demon'], ['knight', 'knave'], ['knight', 'demon']]

In [18]:
# A says 'I am a demon' B says 'I am not a god'
solutions(aX('demon'),complement(bX('god')))

[['knave', 'knight']]

In [19]:
solutions([['god','demon']],[['knave','knave']])

[['god', 'demon'], ['knave', 'demon'], ['demon', 'knave'], ['demon', 'demon']]

## Some possible puzzle sets

In [20]:
# let's define some functions and structures to bring statements together with lists of pairs from those statements
def positiveStatement(option, sets):
    return {"statement":"I am a " + option, "type":option, "pairs": sets}
def negativeStatement(option, sets):
    return {"statement":"I am not a " + option,"type": option, "pairs": sets}
positiveStatement('demon', aX('demon'))

{'statement': 'I am a demon',
 'type': 'demon',
 'pairs': [['demon', 'god'],
  ['demon', 'knight'],
  ['demon', 'knave'],
  ['demon', 'demon']]}

In [21]:
# let's generate a whole bunch of simple statements
aStatements = []
bStatements = []
for i in options:
    aStatements.append(positiveStatement(i,aX(i)))
    aStatements.append(negativeStatement(i,complement(aX(i))))
    bStatements.append(positiveStatement(i,bX(i)))
    bStatements.append(negativeStatement(i,complement(bX(i))))
print(aStatements)

[{'statement': 'I am a god', 'type': 'god', 'pairs': [['god', 'god'], ['god', 'knight'], ['god', 'knave'], ['god', 'demon']]}, {'statement': 'I am not a god', 'type': 'god', 'pairs': [['knight', 'god'], ['knight', 'knight'], ['knight', 'knave'], ['knight', 'demon'], ['knave', 'god'], ['knave', 'knight'], ['knave', 'knave'], ['knave', 'demon'], ['demon', 'god'], ['demon', 'knight'], ['demon', 'knave'], ['demon', 'demon']]}, {'statement': 'I am a knight', 'type': 'knight', 'pairs': [['knight', 'god'], ['knight', 'knight'], ['knight', 'knave'], ['knight', 'demon']]}, {'statement': 'I am not a knight', 'type': 'knight', 'pairs': [['god', 'god'], ['god', 'knight'], ['god', 'knave'], ['god', 'demon'], ['knave', 'god'], ['knave', 'knight'], ['knave', 'knave'], ['knave', 'demon'], ['demon', 'god'], ['demon', 'knight'], ['demon', 'knave'], ['demon', 'demon']]}, {'statement': 'I am a knave', 'type': 'knave', 'pairs': [['knave', 'god'], ['knave', 'knight'], ['knave', 'knave'], ['knave', 'demon']]

In [22]:
def runAllStatements():
    counter = 1
    for a in aStatements:
        for b in bStatements:
            s = solutions(a["pairs"], b["pairs"])
            if (len(s) == 1):
                print("puzzle: " + str(counter))
                counter = counter + 1
                print("A says: " + a["statement"] + ", B says: " + b["statement"])
                print("The solution is that A is a "+ s[0][0] + " and B is a " + s[0][1])
                print("----------------------------------------------------------")
runAllStatements()

puzzle: 1
A says: I am not a god, B says: I am not a god
The solution is that A is a knight and B is a knight
----------------------------------------------------------
puzzle: 2
A says: I am not a god, B says: I am not a knight
The solution is that A is a knight and B is a god
----------------------------------------------------------
puzzle: 3
A says: I am not a god, B says: I am a knave
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 4
A says: I am not a god, B says: I am a demon
The solution is that A is a knight and B is a knave
----------------------------------------------------------
puzzle: 5
A says: I am not a knight, B says: I am not a god
The solution is that A is a god and B is a knight
----------------------------------------------------------
puzzle: 6
A says: I am not a knight, B says: I am not a knight
The solution is that A is a god and B is a god
---------------------------------------------------

## More possible sets

In [23]:
def otherStatement(option, sets):
    return {"statement":"They are a " + option, "type":option, "pairs": sets}

def otherNotStatement(option, sets):
    return {"statement":"They are not a " + option, "type":option, "pairs": sets}

for i in options:
    aStatements.append(otherStatement(i,bX(i)))
    aStatements.append(otherNotStatement(i,complement(bX(i))))
    bStatements.append(otherStatement(i,aX(i)))
    bStatements.append(otherNotStatement(i,complement(aX(i))))

runAllStatements()

In [24]:
def sideStatement(side,sets ):
        return {"statement":"I am " + side, "type":side, "pairs": sets}
def weAreBoth(option,sets):
    return {"statement":"We are both " + option, "type":option, "pairs": sets}

def iAmTheyAre(option1, option2, sets):
    return {"statement":"I am a  " + option1 + " and they are a " + option2, "type":option1, "pairs": sets}

In [25]:
aStatements.append(sideStatement("immortal", fromLists(immortals, options)))
aStatements.append(sideStatement("mortal", fromLists(mortals, options)))
aStatements.append(sideStatement("truth-teller", fromLists(truthfuls, options)))
aStatements.append(sideStatement("liar", fromLists(liars, options)))

bStatements.append(sideStatement("immortal", fromLists(options, immortals)))
bStatements.append(sideStatement("mortal", fromLists(options, mortals)))
bStatements.append(sideStatement("truth-teller", fromLists(options, truthfuls)))
bStatements.append(sideStatement("liar", fromLists(options, liars)))

for i in options:
    aStatements.append(weAreBoth(i + "s", [[i,i]]))
    bStatements.append(weAreBoth(i + "s", [[i,i]]))

aStatements.append(weAreBoth("liars",fromLists(liars,liars)))
bStatements.append(weAreBoth("liars",fromLists(liars,liars)))

aStatements.append(weAreBoth("truthful",fromLists(truthfuls,truthfuls)))
bStatements.append(weAreBoth("truthful",fromLists(truthfuls,truthfuls)))

aStatements.append(weAreBoth("immortal",fromLists(immortals,immortals)))
bStatements.append(weAreBoth("immortal",fromLists(immortals,immortals)))

aStatements.append(weAreBoth("mortals",fromLists(mortals,mortals)))
bStatements.append(weAreBoth("mortals",fromLists(mortals,mortals)))

aStatements.append(iAmTheyAre("mortal","immortal", fromLists(mortals,immortals)))
bStatements.append(iAmTheyAre("mortal","immortal", fromLists(immortals,mortals)))

aStatements.append(iAmTheyAre("liar","truth-teller", fromLists(liars,truthfuls)))
bStatements.append(iAmTheyAre("liar","truth-teller", fromLists(truthfuls,liars)))

for i in options:
    for j in options:        
        aStatements.append(iAmTheyAre(i,j,[[i,j]]))
        bStatements.append(iAmTheyAre(i,j,[[j,i]]))
        
runAllStatements()

puzzle: 1
A says: I am not a god, B says: I am not a god
The solution is that A is a knight and B is a knight
----------------------------------------------------------
puzzle: 2
A says: I am not a god, B says: I am not a knight
The solution is that A is a knight and B is a god
----------------------------------------------------------
puzzle: 3
A says: I am not a god, B says: I am a knave
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 4
A says: I am not a god, B says: I am a demon
The solution is that A is a knight and B is a knave
----------------------------------------------------------
puzzle: 5
A says: I am not a god, B says: I am a  knave and they are a knight
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 6
A says: I am not a god, B says: I am a  demon and they are a knight
The solution is that A is a knight and B is a knave
-----------

In [26]:
bothImmortal = fromLists(immortals,immortals)
print(bothImmortal)

[['god', 'god'], ['god', 'demon'], ['demon', 'god'], ['demon', 'demon']]


In [27]:
bothMortal = fromLists(mortals, mortals)
print(bothMortal)

[['knight', 'knight'], ['knight', 'knave'], ['knave', 'knight'], ['knave', 'knave']]


In [28]:
fromSameDimension = union(bothImmortal, bothMortal)
print(fromSameDimension)

[['god', 'god'], ['god', 'demon'], ['demon', 'god'], ['demon', 'demon'], ['knight', 'knight'], ['knight', 'knave'], ['knave', 'knight'], ['knave', 'knave']]


In [29]:
dimensionStatement = {"statement":"We are from the same dimension", "type":"dimension", "pairs": fromSameDimension}
aStatements.append(dimensionStatement)
bStatements.append(dimensionStatement)

In [30]:
runAllStatements()

puzzle: 1
A says: I am not a god, B says: I am not a god
The solution is that A is a knight and B is a knight
----------------------------------------------------------
puzzle: 2
A says: I am not a god, B says: I am not a knight
The solution is that A is a knight and B is a god
----------------------------------------------------------
puzzle: 3
A says: I am not a god, B says: I am a knave
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 4
A says: I am not a god, B says: I am a demon
The solution is that A is a knight and B is a knave
----------------------------------------------------------
puzzle: 5
A says: I am not a god, B says: I am a  knave and they are a knight
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 6
A says: I am not a god, B says: I am a  demon and they are a knight
The solution is that A is a knight and B is a knave
-----------

In [31]:
differentDimensions = difference(pairs, fromSameDimension)
print(differentDimensions)

[['god', 'knight'], ['god', 'knave'], ['knight', 'god'], ['knight', 'demon'], ['knave', 'god'], ['knave', 'demon'], ['demon', 'knight'], ['demon', 'knave']]


In [32]:
diffDimensionStatement = {"statement":"We are from a different dimension", "type":"dimension", "pairs": differentDimensions}
aStatements.append(diffDimensionStatement)
bStatements.append(diffDimensionStatement)

In [33]:
runAllStatements()

puzzle: 1
A says: I am not a god, B says: I am not a god
The solution is that A is a knight and B is a knight
----------------------------------------------------------
puzzle: 2
A says: I am not a god, B says: I am not a knight
The solution is that A is a knight and B is a god
----------------------------------------------------------
puzzle: 3
A says: I am not a god, B says: I am a knave
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 4
A says: I am not a god, B says: I am a demon
The solution is that A is a knight and B is a knave
----------------------------------------------------------
puzzle: 5
A says: I am not a god, B says: I am a  knave and they are a knight
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 6
A says: I am not a god, B says: I am a  demon and they are a knight
The solution is that A is a knight and B is a knave
-----------

In [34]:
aDemon = aX('demon')
print(aDemon)

[['demon', 'god'], ['demon', 'knight'], ['demon', 'knave'], ['demon', 'demon']]


In [35]:
intersect(aDemon, differentDimensions)

[['demon', 'knight'], ['demon', 'knave']]

In [36]:
def iAmORTheyAre(option1, option2, sets):
    return {"statement":"I am a  " + option1 + " or they are a " + option2, "type":option1, "pairs": sets}


In [37]:
for i in options:
    for j in options:        
        aStatements.append(iAmORTheyAre(i,j, union(aX(i),bX(j))))
        bStatements.append(iAmORTheyAre(i,j, union(aX(j),bX(i))))

In [38]:
runAllStatements()

puzzle: 1
A says: I am not a god, B says: I am not a god
The solution is that A is a knight and B is a knight
----------------------------------------------------------
puzzle: 2
A says: I am not a god, B says: I am not a knight
The solution is that A is a knight and B is a god
----------------------------------------------------------
puzzle: 3
A says: I am not a god, B says: I am a knave
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 4
A says: I am not a god, B says: I am a demon
The solution is that A is a knight and B is a knave
----------------------------------------------------------
puzzle: 5
A says: I am not a god, B says: I am a  knave and they are a knight
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 6
A says: I am not a god, B says: I am a  demon and they are a knight
The solution is that A is a knight and B is a knave
-----------

In [39]:
union(aX('demon'),fromLists(options,mortals))

[['demon', 'god'],
 ['demon', 'knight'],
 ['demon', 'knave'],
 ['demon', 'demon'],
 ['god', 'knight'],
 ['god', 'knave'],
 ['knight', 'knight'],
 ['knight', 'knave'],
 ['knave', 'knight'],
 ['knave', 'knave']]

In [40]:
for i in options:
    aStatements.append(iAmTheyAre(i, "mortal", fromLists([i],mortals)))
    aStatements.append(iAmORTheyAre(i,"mortal", union(aX(i),fromLists(options,mortals))))
    aStatements.append(iAmTheyAre(i, "immortal", fromLists([i],immortals)))
    aStatements.append(iAmORTheyAre(i,"immortal", union(aX(i),fromLists(options,immortals))))
    ##
    bStatements.append(iAmTheyAre(i, "mortal", fromLists(mortals,[i])))
    bStatements.append(iAmORTheyAre(i,"mortal", union(bX(i),fromLists(options,mortals))))
    bStatements.append(iAmTheyAre(i, "immortal", fromLists(immortals,[i])))
    bStatements.append(iAmORTheyAre(i,"immortal", union(bX(i),fromLists(options,immortals))))
    
    

In [41]:
runAllStatements()

puzzle: 1
A says: I am not a god, B says: I am not a god
The solution is that A is a knight and B is a knight
----------------------------------------------------------
puzzle: 2
A says: I am not a god, B says: I am not a knight
The solution is that A is a knight and B is a god
----------------------------------------------------------
puzzle: 3
A says: I am not a god, B says: I am a knave
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 4
A says: I am not a god, B says: I am a demon
The solution is that A is a knight and B is a knave
----------------------------------------------------------
puzzle: 5
A says: I am not a god, B says: I am a  knave and they are a knight
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 6
A says: I am not a god, B says: I am a  demon and they are a knight
The solution is that A is a knight and B is a knave
-----------

In [42]:
sameType = []
for i in options:
    sameType.append([i,i])
sameTypeStatement ={"statement":"We are the same type", "type":"same", "pairs":sameType }
aStatements.append(sameTypeStatement)
bStatements.append(sameTypeStatement)


In [43]:
runAllStatements()

puzzle: 1
A says: I am not a god, B says: I am not a god
The solution is that A is a knight and B is a knight
----------------------------------------------------------
puzzle: 2
A says: I am not a god, B says: I am not a knight
The solution is that A is a knight and B is a god
----------------------------------------------------------
puzzle: 3
A says: I am not a god, B says: I am a knave
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 4
A says: I am not a god, B says: I am a demon
The solution is that A is a knight and B is a knave
----------------------------------------------------------
puzzle: 5
A says: I am not a god, B says: I am a  knave and they are a knight
The solution is that A is a knight and B is a demon
----------------------------------------------------------
puzzle: 6
A says: I am not a god, B says: I am a  demon and they are a knight
The solution is that A is a knight and B is a knave
-----------

The solution is that A is a demon and B is a knave
----------------------------------------------------------
puzzle: 291
A says: We are the same type, B says: I am a  knave and they are a demon
The solution is that A is a knave and B is a demon
----------------------------------------------------------
puzzle: 292
A says: We are the same type, B says: I am a  demon and they are a knave
The solution is that A is a demon and B is a knave
----------------------------------------------------------
puzzle: 293
A says: We are the same type, B says: I am a  knave and they are a immortal
The solution is that A is a knave and B is a demon
----------------------------------------------------------
puzzle: 294
A says: We are the same type, B says: I am a  demon and they are a mortal
The solution is that A is a demon and B is a knave
----------------------------------------------------------
